# Teori  

## Skalering  

[först pratade vi om standardisering och normalisering - jag missade när vi ska använda någon av dem]  

Om vi reguljariserar så måste vi standardisera, dvs att vi måste ha $\beta_i$ på samma skala. Även de tillfällen då noll ska betyda "ingenting" så behövs ofta skalering.  

Y skalerar vi oftast inte eftersom vi vill behålla storleken på responsen (Y) och inte behöva konvertera fram och tillbaka. Men om vi klassificerar och får in sannolikheter så behöver vi skalera eftersom de ska summera till 1 (ett).  

Det finns inneboende risk med att skalera. Om datan inte är ~N så [MISSADE NÅGOT HÄR] ser vi inte skillnader.  

Oftast beror det på föreskriven metid, alltså vilken regression som används, som säger om vi ska skalera eller inte.  

## Gradient descent

OLS är ett specialfall av maximum-likelihood metoden (inte korrekt skriven nedqan):  

$ max\bold{CB} \approx \prod P \prod 1-P $  

Symbolen $\prod$ är samma sak som $\sum$ fast för multiplikation i stället för addition.  

Då fungerar inte längre formeln för våra $\beta$ som vi hade för linjär regression. Det är inte alltid så att vi kan härleda ett b för koefficienterna. I denna kostnadsfunktion finns inget krav på linjaritet. Vi måste alltså hitta ett annat sätt att optimera över konstadsfunktionen. Det finns många sätt men vad vi framför allt för är att vi använder gradient descent - en iterativ metod (vår kostnadsfunktion har nu en annan form).  Den fungerar så bra att vi nästan gett upp alla andra metoder.  

Steglängden är viktig. Om ytan är "knagglig" så hoppar vi omkring massa (instabilt). Så vad gör vi? Vi kan använda **SGD - stochastic gradient descent**. Då väljer vi bara en slumpmässig punkt i varje iteration i stället för hela stickprovet. Den kan ju då vara var som helst. Nu konvergerar inte metoden längre. Men vi måste däremot ha ett stoppvillkor för att den inte ska fortsätta i all evighet. Vad denna metod gör är att den inte fastnar i lokala minimum. Sen har vi även **mini-batch gradient descent** så väljer vi en slumpmässig delmängd av punkterna och räknar gradienten på. En iterativ metod betyder att vi måste ha stoppvillkor och varje steg vi tar kallas för en epok. Det som är bra att veta är att iterativa metoder kan bli kaotiska.  

$ x_{n+1} = r (1 - x_n) $  

3.58 är oftast värdet då det blir kaotiskt. Det är en fraktal så de får olika mönster beroende på vilka värden vi välker. Det finns inga analytiska sätt att se om det bryter ut i kaos.  

OLS: 0(np²), n - stickprovets storlek, p - antalet dimensioner  
SGD: 0(n), därav är SGD är alltid linjär  

Vi behöver justera några parametrar för att SGD ska bli bra:  
- steglängden bör förändras (adaptive gradient descent)  
- moment (tänk att vanlig GD kan fastna i en liten svacka men med momentum kommer den över den och fastnar i den verkliga minimumet) (ADAM - adaptive moment estimation)  

# Code - L3-Gradient_descent

## (Batch) Gradient Descent  

Missade. 

## Stochastic Gradient Descent  



In [ ]:
def learning_rate_schedule(t):
    """Decrease learning rate as training progresses"""
    return t0/(t+t1)


[...]
eta = learning_rate_schedule(epoch*m+i)
[...]

Nu ändrar vi hur vi hanterar vår learning rate, den blir adaptiv. Vid olika tillfällen ska vi ändra vår learning rate. Mindre ju fler epoker vi har och mindre ju fler dimensioner vi har. 

SGD kan hoppa ur lokala minimum men även fastna. Tidigare metod kommer fastna i ett lokalt minimum. Med tiden kommer den täcka alla punkter. 

För mini batch gradient descent så kommer 

# Klassificering - L4-Classification

Nu kan vi inte längre använda error metrics, i stället tittar vi på klassificeringsrapporter. Det handlar om typ-1 och typ-2 fel (falskt positiv, falskt negativ). De mäter hur bra en klassificering är. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)

Vi har alltså plattat till bilden till en lång vektor för varje pixel med 64 parametrar. Med KNN satt till 1 kommer den att titta på närmsta punkten, i stort sett jämföra pixel för pixel och klassificera efter närmsta. Vi mäter alltså avståndet till närmsta vektor. 

Alla värden är pixelvärden mellan 0-255 så det finns ingen mening med att standardisera eller centrera till noll (eftersom noll innebär att det inte finns något där). Datan vi har är alltså redan begränsad. 

Rapporten (vi kommer gå igenom denna mer noggrant senare): 

support - Hur många av varje exempel finns med i resultatet. Hur många förekommer.   
precision - om hög precision så har vi inte typ-1 fel (false positives). Vi gissar t ex aldrig på att en sexa är något annat (med resultat 1.0).  

Accuracy motsvarar vår RMSE, hur många hade vi rätt på. Vi får 98 % rätt på att gissa 

Så länge vi har få klasser så är KNN bland det bästa vi kan använda. Anledningen är att om vi ökar dimensionerna så blir det euclidiska avståndet längre hela tiden. Med fler dimensioner så blir summan bara större och större, då slutar KNN att fungera. Då finns inget nära längre, varje punkt blir isolerad. The Curse of Dimensionality. I allmänhet är det så, det blir mer och mer tomrum och tomma dimensioner och avstånden ökar. Det är också en av anledningarna till att använda L1-norm i stället för euclidiskt avstånd. Tänk en rätvinklig triangel och Pythagoras. Där är kateterna alltid kortare än hypotenusan. L1-normen är alltså kateterna och L2-normen är hypotenusan. 

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

cm = confusion_matrix(Y_test, Y_knn)
ConfusionMatrixDisplay(cm).plot()

Mycket mer intuitivt!  

Diagonalen är support.  

Vi kan se att vi t ex tre gånger gissat på en etta men det var faktiskt en åtta.  

In [ ]:
for k in range(1, 50):
    model_KNN = KNeighborsClassifier(n_neighbors=k)
    model_KNN.fit(scaled_X_train, Y_train)
    Y_hat = model_KNN.predict(scaled_X_test)
    error_list += [1-accuracy_score(Y_test, Y_hat)]

Här testar vi 1-50 varianter av K. En form av hyperparameteroptimering. Här ser vi att vår modell blir bara sämre och sämre ju fler K vi har. Däremot finns en dipp vid vad som ser ut att vara 3. Om vi då kör om allting på 3 i stället så får vi bättre resultat. 